<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/distilbert_clpsych15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [ ]:
notebook_login()

In [ ]:
model_id = "migueladarlo/distilbert-depression-base"
##dataset_id = "ag_news"
# replace the value with your model: ex <hugging-face-user>/<model-name>
repository_id = "geee05/distilbert-clpsych15"

In [ ]:
import pandas as pd
from datasets import ClassLabel, Dataset
data=pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
test=pd.read_csv('/content/DAIC_test_3sp.csv')
#label_test = ClassLabel(num_classes=2, names=[0, 1])

data.rename(columns = {'Group':'label'}, inplace = True)
test.rename(columns = {'Group':'label'}, inplace = True)

dataset = Dataset.from_dict(data)
test_dataset=Dataset.from_dict(test)

dataset = dataset.class_encode_column("label")
test_dataset = test_dataset.class_encode_column("label")

Stringifying the column:   0%|          | 0/10264 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10264 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1914 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [ ]:
data['label'].value_counts()

0    5132
1    5132
Name: label, dtype: int64

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:

# Training and testing datasets
train_dataset = dataset
test_dataset = test_dataset
# Validation dataset
##val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    #print(tokenizer(batch["text"], padding=True, truncation=True, max_length=256)['input_ids'])
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
##val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
# Set dataset format
train_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
#val_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
test_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features['label'].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 2
the labels: ['0', '1']


In [ ]:
# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate


  
   learning_rate=2e-5,


In [ ]:
import numpy as np
# Model
model = DistilBertForSequenceClassification.from_pretrained(model_id, config=config)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)
from datasets import load_metric
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


/content/geee05/distilbert-clpsych15 is already a clone of https://huggingface.co/geee05/distilbert-clpsych15. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.292500,1.565494,0.646813,0.294363
2,0.118100,1.733338,0.663009,0.262857
3,0.062900,1.733338,0.663009,0.262857


TrainOutput(global_step=3849, training_loss=0.14226057370517292, metrics={'train_runtime': 849.0351, 'train_samples_per_second': 36.267, 'train_steps_per_second': 4.533, 'total_flos': 2027214835332096.0, 'train_loss': 0.14226057370517292, 'epoch': 3.0})

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 1.565494179725647,
 'eval_accuracy': 0.6468129571577848,
 'eval_f1': 0.2943632567849687,
 'eval_runtime': 13.2751,
 'eval_samples_per_second': 144.18,
 'eval_steps_per_second': 18.079,
 'epoch': 3.0}